# Molecule Generation and Property Calculation

In this notebook, you will get some hands-on experience with two cheminformatics techniques that medicinal chemists use. 
We will be performing a "positional analogue scan" of a moleule by doing a "nitrogen walk" in a ring system.

This lab is based heavily on [a blog post](https://practicalcheminformatics.blogspot.com/2020/04/positional-analogue-scanning.html) by [Dr. Patrick Walters](https://www.linkedin.com/in/wpwalters/), a professional chemist and Chief Data Officer of Relay Therapeutics.
Dr. Walters has a blog, [Practical Cheminformatics](https://practicalcheminformatics.blogspot.com/) that he updates with evaluations and code implementations of Cheminformatics concepts. 
Dr. Walters also provides many code examples for you to follow along with the blog posts if you so wish.
It is an excellent resource for learning about cheminformatics!

## Generating Molecules: Positional Analogue Scanning
In medicinal chemistry and cheminformatics, one task scientists have is to generate and test new molecules.
With the Positional Analogue Scanning (PAS) technique, one takes a "lead" molecule and a substituent of interest and "walks" the substituent around the molecule, placing it in different positions in the molecule.
The generated structures can then be evaluated in different ways to see if any of them are promising candidates for further testing.

In this lab, we will perform a type of PAS called a "nitrogen walk". 
In a nitrogen walk, aromatic carbons in ring systems are replaced by nitrogen atoms.
We will use the function for a nitrogen walk provided by Dr. Walters in his blog post.

**Exercise** - Explain what each line of the function does by adding comments to the function.

In [ ]:
from rdkit import Chem
from itertools import combinations

def nitrogen_walk(mol_in, num_N=1):
    out_mol_list = []
    used = set()
    aromatic_cH = Chem.MolFromSmarts("[cH]")
    match_atms = [x[0] for x in mol_in.GetSubstructMatches(aromatic_cH)]
    n_combos = combinations(match_atms, num_N)
    for combo in n_combos:
        new_mol = Chem.RWMol(mol_in)
        for idx in combo:
            atm = new_mol.GetAtomWithIdx(idx)
            atm.SetAtomicNum(7)
        smi = Chem.MolToSmiles(new_mol)
        if smi not in used:
            used.add(smi)
            out_mol_list.append(new_mol)
    return out_mol_list

To understand what is happening, let's apply this nitrogen walk to benzene and to aniline, which we looked at in the last notebook.

In [ ]:
# make benzene and aniline molecules
benzene = Chem.MolFromSmiles("c1ccccc1")
aniline = Chem.MolFromSmiles("c1ccccc1N")

In [ ]:
# create analogues
benzene_analogues = nitrogen_walk(benzene, 1)
aniline_analogues = nitrogen_walk(aniline, 1)

If we visualize the molecules after our nitrogen walk, we will see which atoms matched our aromatic carbon pattern.

In [ ]:
benzene

In [ ]:
aniline

In [ ]:
# visualize
from rdkit.Chem import Draw

benzene_molecules = [ benzene ] + benzene_analogues
labels = [ "Benzene" ] + [  f"Analogue {i}" for i in range(1, len(benzene_molecules))] 
Draw.MolsToGridImage(benzene_molecules, molsPerRow=4, subImgSize=(200, 200), legends=labels)

In [ ]:
## Repeat to draw aniline analogues



**Chemistry Question** - Why does benzene only have one analogue when there are six aromatic nitrogens? Why does nitrobenzene have three analogues?


**Exercise** - Try generating more analogues by adding more nitrogens to the rings.


## Mearsuring Drug-Likeness: QED

Once molecules are generated, they will be analyzed further to evaluate appropriateness for a specific purpose. 
In his blog post, Dr. Walters give the following possibilities of follow up studies:

* Evaluating the impact of small changes in chemical structure on conformation.   By comparing conformational ensembles between two related molecules we can see which changes will, and will not, impact conformation.  
* 
Examine how positional analogs impact a molecule's electrostatic potential
* 
Docking the analogs and examining which changes preserve key interactions and which ones make new interaction
*  
Carrying out free energy calculations to predict changes that will potentially improve a compound's binding affini

Some of these, like electrostsic potential, could be calculated using QM software like Psi4 that we learned about in an earlier lab.

For the purposes of our exercise, we will evaluate the generated molecule's "drug-likeness" using a method called "Quantitative Estimate of Druglikeness" or QED. 
QED was first proposed in a [paper published in 2012](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3524573/#R27). 
QED is simply a measure of how similar a molecule is to known approved drugs.
QED is far from the only measurement for evaluating molecules, and it is important to remember that it
doesn't indicate a molecule's effectiveness as a drug for a particular purpose.

The formula for QED is:

$$
QED = \exp \left( -\frac{1}{n} \sum_{i=1}^{n} \ln d_i \right)
$$

Where $n$ corresponds to a particular molecular descriptor and $d$ is a desirability function for each descriptor.
The "desirability functions" were derived based on observations of a set of known drug molecules.

The molecular properties (chemical descriptors) used for QED by the original paper were:

1. Molecular weight
2. Octanol-watter partition coefficient (ALOGP)
3. Number of hydrogen bond donors
4. Number of hydrogen bond acceptors
5. Molecular polar surface area
6. Number of rotatable bonds
7. Number of aromatic rings
8. Number of structural alerts

While we could pull most of these descriptors from RDKit ourselves to implement a QED calculation, it is beyond
the scope of this lab.
We will use RDKit's built-in [QED function](https://www.rdkit.org/docs/source/rdkit.Chem.QED.html) to calculate the "drug-likeness" of our analogues.

In [ ]:
import pandas as pd

from rdkit.Chem import QED

In [ ]:
df = pd.DataFrame(columns=["mol", "smiles", "qed"])

In [ ]:
df["mol"] = benzene_molecules
df["smiles"] = df["mol"].apply(Chem.MolToSmiles)

In [ ]:
df.head()

In [ ]:
df["qed"] = df["mol"].apply(QED.qed)

In [ ]:
df.head()

For visualizing molecules in a dataframe, it is convenient to use the `mols2grid` package (included in your environment).

In [ ]:
import mols2grid

# display our df. The subset argument controls what information will show up in the grid
# we will display the SMILES, molecule image, and calculated QED score.
mols2grid.display(df, smiles_col="smiles", subset=["smiles", "img", "qed"], size=(200,200))

In [ ]:
## Repeat for aniline analogues


## Exercise - Generate Molecules and Evaluate QED
For this next section, you will be performing a nitrogen walk for a lead molecule candidate.
You should generate analogues then evaluate the QED of the generated molecules.
Generate analogues that have 1, 2, or 3 nitrogens.

Put the results in a dataframe and pick the five molecules with the highest QED scores.
Cross-reference your molecules in PubChem. 
Do any of them correspond to real drugs?

In [ ]:
lead_molecule = "CCc1cc(N)cc(N)c1-c1ccc(Cl)cc1"
mol = Chem.MolFromSmiles(lead_molecule)

mol